In [1]:
spark

Waiting for a Spark session to start...

Spark application_1540458187951_76576: Some(http://gw02.itversity.com:4041)

In [2]:
sc.getConf.getAll.filter(_._2.contains("/proxy/"))(0)._2

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1540458187951_76576

In [3]:
def getType(o: Any) = o.getClass.getCanonicalName

getType: (o: Any)String


In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getenv("HOME").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home+"/dataSets/spark-guide/activity-data/"
val path1 = hdfs_home+"/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet/"

path = /user/kranthidr/dataSets/spark-guide/activity-data/
path1 = /user/kranthidr/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet/


/user/kranthidr/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet/

In [6]:
// in Scala
val static = spark.read.json(path)
val dataSchema = static.schema

static = [Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]
dataSchema = StructType(StructField(Arrival_Time,LongType,true), StructField(Creation_Time,LongType,true), StructField(Device,StringType,true), StructField(Index,LongType,true), StructField(Model,StringType,true), StructField(User,StringType,true), StructField(gt,StringType,true), StructField(x,DoubleType,true), StructField(y,DoubleType,true), StructField(z,DoubleType,true))


StructType(StructField(Arrival_Time,LongType,true), StructField(Creation_Time,LongType,true), StructField(Device,StringType,true), StructField(Index,LongType,true), StructField(Model,StringType,true), StructField(User,StringType,true), StructField(gt,StringType,true), StructField(x,DoubleType,true), StructField(y,DoubleType,true), StructField(z,DoubleType,true))

In [7]:
// COMMAND ----------

// in Scala
val streaming = spark.readStream.schema(dataSchema)
  .option("maxFilesPerTrigger", 1).json(path)

streaming = [Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]


[Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]

In [11]:
// COMMAND ----------

// in Scala
val activityCounts = streaming.groupBy("gt").count()

activityCounts = [gt: string, count: bigint]


lastException: Throwable = null


[gt: string, count: bigint]

In [12]:
// COMMAND ----------

spark.conf.set("spark.sql.shuffle.partitions", 5)

In [13]:
// COMMAND ----------

// in Scala
val activityQuery = activityCounts.writeStream.queryName("activity_counts")
  .format("memory").outputMode("complete")
  .start()

activityQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4e0b4a69


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4e0b4a69

In [14]:
// COMMAND ----------

//activityQuery.awaitTermination()

Name: Syntax Error.
Message: 
StackTrace: 

In [15]:
// COMMAND ----------

spark.streams.active

[org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4e0b4a69]

In [16]:
// COMMAND ----------

// in Scala
for( i <- 1 to 5 ) {
    spark.sql("SELECT * FROM activity_counts").show()
    Thread.sleep(1000)
}

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|12309|
|     stand|11384|
|stairsdown| 9365|
|      walk|13256|
|  stairsup|10452|
|      null|10449|
|      bike|10796|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|12309|
|     stand|11384|
|stairsdown| 9365|
|      walk|13256|
|  stairsup|10452|
|      null|10449|
|      bike|10796|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|12309|
|     stand|11384|
|stairsdown| 9365|
|      walk|13256|
|  stairsup|10452|
|      null|10449|
|      bike|10796|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|24619|
|     stand|22769|
|stairsdown|18729|
|      walk|26512|
|  stairsup|20905|
|      null|20896|
|      bike|21593|
+----------+-----+



In [17]:
// COMMAND ----------

// in Scala
import org.apache.spark.sql.functions.expr
val simpleTransform = streaming.withColumn("stairs", expr("gt like '%stairs%'"))
  .where("stairs")
  .where("gt is not null")
  .select("gt", "model", "arrival_time", "creation_time")
  .writeStream
  .queryName("simple_transform")
  .format("memory")
  .outputMode("append")
  .start()

simpleTransform = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6e6ee1ea


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6e6ee1ea

In [18]:
// COMMAND ----------

// in Scala
val deviceModelStats = streaming.cube("gt", "model").avg()
  .drop("avg(Arrival_time)")
  .drop("avg(Creation_Time)")
  .drop("avg(Index)")
  .writeStream.queryName("device_counts").format("memory").outputMode("complete")
  .start()

deviceModelStats = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@1e74cac7


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@1e74cac7

In [19]:
// COMMAND ----------

// in Scala
// val historicalAgg = static.groupBy("gt", "model").avg()

// val deviceModelStats_aggr = streaming.drop("Arrival_Time", "Creation_Time", "Index")
//   .cube("gt", "model").avg()
//   .join(historicalAgg, Seq("gt", "model"))
//   .writeStream.queryName("device_counts_aggr").format("memory").outputMode("complete")
//   .start()

//Taking too long

Name: Syntax Error.
Message: 
StackTrace: 

In [20]:
// COMMAND ----------

// in Scala
// Subscribe to 1 topic
// val ds1 = spark.readStream.format("kafka")
//   .option("kafka.bootstrap.servers", "host1:port1,host2:port2")
//   .option("subscribe", "topic1")
//   .load()

Name: Syntax Error.
Message: 
StackTrace: 

In [21]:
// Subscribe to multiple topics
// val ds2 = spark.readStream.format("kafka")
//   .option("kafka.bootstrap.servers", "host1:port1,host2:port2")
//   .option("subscribe", "topic1,topic2")
//   .load()

Name: Syntax Error.
Message: 
StackTrace: 

In [22]:
// Subscribe to a pattern of topics
// val ds3 = spark.readStream.format("kafka")
//   .option("kafka.bootstrap.servers", "host1:port1,host2:port2")
//   .option("subscribePattern", "topic.*")
//   .load()

Name: Syntax Error.
Message: 
StackTrace: 

In [23]:
// COMMAND ----------

// in Scala
// ds1.selectExpr("topic", "CAST(key AS STRING)", "CAST(value AS STRING)")
//   .writeStream.format("kafka")
//   .option("checkpointLocation", "/to/HDFS-compatible/dir")
//   .option("kafka.bootstrap.servers", "host1:port1,host2:port2")
//   .start()

Name: Syntax Error.
Message: 
StackTrace: 

In [24]:
// ds1.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
//   .writeStream.format("kafka")
//   .option("kafka.bootstrap.servers", "host1:port1,host2:port2")
//   .option("checkpointLocation", "/to/HDFS-compatible/dir")\
//   .option("topic", "topic1")
//   .start()

Name: Syntax Error.
Message: 
StackTrace: 

In [26]:
// COMMAND ----------

//in Scala
// datasetOfString.write.foreach(new ForeachWriter[String] {
//   def open(partitionId: Long, version: Long): Boolean = {
//     // open a database connection
//   }
//   def process(record: String) = {
//     // write string to connection
//   }
//   def close(errorOrNull: Throwable): Unit = {
//     // close the connection
//   }
// })

Name: Syntax Error.
Message: 
StackTrace: 

In [27]:
// COMMAND ----------

// in Scala
// val socketDF = spark.readStream.format("socket")
//   .option("host", "localhost").option("port", 9999).load()

Name: Syntax Error.
Message: 
StackTrace: 

In [28]:
// COMMAND ----------

//activityCounts.format("console").write()

Name: Syntax Error.
Message: 
StackTrace: 

In [29]:
// COMMAND ----------

// in Scala
activityCounts.writeStream.format("memory").queryName("my_device_table")

org.apache.spark.sql.streaming.DataStreamWriter@74112522

In [30]:
// COMMAND ----------

// in Scala
import org.apache.spark.sql.streaming.Trigger

activityCounts.writeStream.trigger(Trigger.ProcessingTime("100 seconds"))
  .format("console").outputMode("complete").start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@e82711d

-------------------------------------------
Batch: 0
-------------------------------------------
+----------+-----+
|        gt|count|
+----------+-----+
|       sit|12309|
|     stand|11384|
|stairsdown| 9365|
|      walk|13256|
|  stairsup|10452|
|      null|10449|
|      bike|10796|
+----------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+----------+-----+
|        gt|count|
+----------+-----+
|       sit|24619|
|     stand|22769|
|stairsdown|18729|
|      walk|26512|
|  stairsup|20905|
|      null|20896|
|      bike|21593|
+----------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+----------+-----+
|        gt|count|
+----------+-----+
|       sit|36929|
|     stand|34154|
|stairsdown|28094|
|      walk|39768|
|  stairsup|31357|
|      null|31343|
|      bike|32390|
+----------+-----+



In [31]:
// COMMAND ----------

// in Scala
import org.apache.spark.sql.streaming.Trigger

activityCounts.writeStream.trigger(Trigger.Once())
  .format("console").outputMode("complete").start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@28a6f65b

-------------------------------------------
Batch: 0
-------------------------------------------
+----------+-----+
|        gt|count|
+----------+-----+
|       sit|12309|
|     stand|11384|
|stairsdown| 9365|
|      walk|13256|
|  stairsup|10452|
|      null|10449|
|      bike|10796|
+----------+-----+



In [32]:
// COMMAND ----------

// in Scala
case class Flight(DEST_COUNTRY_NAME: String, ORIGIN_COUNTRY_NAME: String,
  count: BigInt)

defined class Flight


In [33]:
val dataSchema = spark.read.parquet(path1).schema

dataSchema = StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,true))


StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,true))

In [34]:
val flightsDF = spark.readStream.schema(dataSchema).parquet(path1)

flightsDF = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [35]:
val flights = flightsDF.as[Flight]

flights = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [36]:
def originIsDestination(flight_row: Flight): Boolean = {
  return flight_row.ORIGIN_COUNTRY_NAME == flight_row.DEST_COUNTRY_NAME
}

originIsDestination: (flight_row: Flight)Boolean


In [37]:
flights.filter(flight_row => originIsDestination(flight_row))
  .groupByKey(x => x.DEST_COUNTRY_NAME).count()
  .writeStream.queryName("device_counts_ds").format("memory").outputMode("complete")
  .start()


// COMMAND ----------

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5fbe457

In [38]:
for( i <- 1 to 5 ) {
    spark.sql("SELECT * FROM device_counts_ds").show()
    Thread.sleep(1000)
}

+-------------+--------+
|        value|count(1)|
+-------------+--------+
|United States|       1|
+-------------+--------+

+-------------+--------+
|        value|count(1)|
+-------------+--------+
|United States|       1|
+-------------+--------+

+-------------+--------+
|        value|count(1)|
+-------------+--------+
|United States|       1|
+-------------+--------+

+-------------+--------+
|        value|count(1)|
+-------------+--------+
|United States|       1|
+-------------+--------+

+-------------+--------+
|        value|count(1)|
+-------------+--------+
|United States|       1|
+-------------+--------+

